In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Capstone workaround").getOrCreate()

In [3]:
data_df = spark.read.csv("../data/imdb/data.tsv", header=True, sep="\t")

In [4]:
data_df.count()

31738509

In [5]:
data_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)



In [9]:
data_df.select("*").where("titleId = 'tt0114709' and title = 'Toy Story'").show()

+---------+--------+---------+------+--------+-----------+--------------+---------------+
|  titleId|ordering|    title|region|language|      types|    attributes|isOriginalTitle|
+---------+--------+---------+------+--------+-----------+--------------+---------------+
|tt0114709|      11|Toy Story|    NO|      \N|imdbDisplay|            \N|              0|
|tt0114709|      13|Toy Story|    AE|      \N|imdbDisplay|            \N|              0|
|tt0114709|      15|Toy Story|    UY|      \N|         \N|dubbed version|              0|
|tt0114709|      16|Toy Story|    DK|      \N|imdbDisplay|            \N|              0|
|tt0114709|      19|Toy Story|    \N|      \N|   original|            \N|              1|
|tt0114709|       1|Toy Story|    IN|      ta|imdbDisplay|            \N|              0|
|tt0114709|      23|Toy Story|    AR|      \N|         \N|            \N|              0|
|tt0114709|      26|Toy Story|    SE|      \N|imdbDisplay|            \N|              0|
|tt0114709